In [ ]:
#importing dependencies

In [107]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import urllib.request 

In [3]:
import pandas as pd

In [14]:
import re

In [ ]:
 import nltk
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [77]:
import numpy as np

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [105]:
file = pd.read_excel('Input.xlsx')

In [ ]:
#extracting content and titles from urls

In [104]:
def extract_title_content(df):
    ''' Extracting title and content from urls'''
    from bs4 import BeautifulSoup
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
      }

    URL = df.URL
    r = requests.get(URL, headers= headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    text = soup.find_all(text=True)
    #set([t.parent.name for t in text])

    blacklist = [
        '[document]','noscript','header','html',
        'meta','head', 'input','script','a',
     'article','aside','body','button','div',
     'figcaption','footer','form', 'h1',
     'head', 'header', 'html', 'li',
     'link', 'meta',  'pre', 'script',
     'span', 'strong', 'style', 'time', 'ul']
    for t in text:
        if t.parent.name not in blacklist:
            if t.parent.name == 'title':
                title=t
            else:
                content = t

    df['Title'] = title
    df['content'] =content
    return df

In [108]:
file=file.apply(extract_title_content, axis=1)

In [9]:
# saving file with content and title
file.to_csv('file.csv', index=False)

In [5]:
# creating list of Stopwords
with open ("StopWords_Generic.txt", 'r') as f:
    stop = f.readlines()
    stopwords =[]
    for word in stop:
        stopwords.append(word)
    

In [13]:
# loading saved file in earlier steps
file=pd.read_csv('file.csv')

In [ ]:
#creating Postive Negative words Dictionary

In [7]:
dic = pd.read_csv('MasterDictionary.csv')

In [8]:
positive= dic.Word[dic.Positive!=0].to_list()
negative = dic.Word[dic.Negative!=0].to_list()

In [9]:
new_dic=dict()
new_dic['Positive'] = [word for word in positive if word not in stopwords]
new_dic['Negative'] =[word for word in negative if word not in stopwords ]


In [128]:
def calculating_scores(row):
    '''Analysis of text'''
    tokens = word_tokenize(row.content)
    total_unclean_words = len(tokens)
    tokens= [x.upper() for x in tokens]
    #removing punctuatuion
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    pos=0; neg=0 ; ch =0
    for token in tokens:
        ch += len(token)
        if token in new_dic['Positive']:
            pos +=1
        elif  token in new_dic['Negative']:
            neg -=1  
    neg = neg * (-1)
    polarity = (pos-neg)/ ((pos + neg) + 0.000001) 
    cont = row.content
    word = cont.split()
    cont_clean=[token for token in tokens if token not in stopwords]
    tot_clean_words = len(cont_clean)
    
    sentence = sent_tokenize(row.content)
    tot_sent = len(sentence)
    av_sent_len = tot_clean_words/(tot_sent + 0.000001)
    
    count_complexWord, avg_syllablePerWord = count_complex(row.content)
    perc_complex = count_complexWord / (tot_clean_words+ 0.000001)
    fogIndex = 0.4 * (av_sent_len + perc_complex)
    
    subjectivity = (pos + neg)/ (tot_clean_words + 0.000001)
    
    avg_wordPerSent = total_unclean_words/(tot_sent + 0.000001)
    
    av_word_length = ch/(total_unclean_words+ 0.000001)
    count_perspronoun = count_pronoun(row.content)
    
    
    row['POSITIVE SCORE'] = pos
    row['NEGATIVE SCORE'] =  neg
    row['POLARITY SCORE'] =  polarity
    row['SUBJECTIVITY SCORE'] =  subjectivity
    row['AVG SENTENCE LENGTH'] =  av_sent_len
    row['PERCENTAGE OF COMPLEX WORDS'] =  perc_complex
    row['FOG INDEX'] =  fogIndex
    row['AVG NUMBER OF WORDS PER SENTENCE'] =  avg_wordPerSent
    row['COMPLEX WORD COUNT'] =  count_complexWord
    row['WORD COUNT'] =  tot_clean_words
    row['SYLLABLE PER WORD'] =  avg_syllablePerWord
    row['PERSONAL PRONOUNS'] =  count_perspronoun
    row['AVG WORD LENGTH'] =  av_word_length
    
    return row     

In [122]:
def count_complex(text):
    '''counting syllables and complex words'''
    complx= []
    vowel_l = []
    tokens = word_tokenize(text)
    tokens= [x.lower() for x in tokens]
    for token in tokens:
        vowel = len(re.findall("e|a|i|o|u", token))-len(re.findall("\w+ed|s$", token))
        vowel_l.append(vowel)
        if vowel > 2 :
            complx.append(token)
#     print(tokens)
#     print(vowel,complx)        
    return len(complx) , np.mean(vowel_l)      

In [123]:
def count_pronoun(text):
    '''counting personal pronoun (excluding country US)'''
    tot = 0
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronoun = pronounRegex.findall(text)
    total = len(pronoun)
      
    return total
        

In [124]:
file.head()

,URL_ID,URL,Title,content
0,1.0,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,With the use of AI and Deep Learning technolog...
1,2.0,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,"By this system, if it is implemented in a supe..."
2,3.0,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,It’s too early to tell how these AI applicatio...
3,4.0,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,"Understanding exactly how data is ingested, an..."
4,5.0,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,"So, what we have understood is AI has penetrat..."


In [129]:
new = file.apply(calculating_scores, axis=1)

In [130]:
new.head()

,URL_ID,URL,Title,content,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1.0,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,With the use of AI and Deep Learning technolog...,0,0,0.000000,0.000000,49.999950,0.200000,20.079980,49.999950,10,50,1.780000,1,4.920000
1,2.0,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,"By this system, if it is implemented in a supe...",1,0,0.999999,0.007752,64.499968,0.093023,25.837196,64.499968,12,129,1.341085,2,4.062015
2,3.0,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,It’s too early to tell how these AI applicatio...,1,0,0.999999,0.016667,29.999985,0.183333,12.073327,29.999985,11,60,1.666667,0,4.566667
3,4.0,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,"Understanding exactly how data is ingested, an...",5,1,0.666667,0.013015,32.928569,0.245119,13.269475,32.928569,113,461,1.713666,1,4.867679
4,5.0,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,"So, what we have understood is AI has penetrat...",2,1,0.333333,0.063830,15.666661,0.191489,6.343260,15.666661,9,47,1.553191,2,4.021277


In [131]:
new.to_csv('output_Rashi.csv')